In [40]:
import os
import email
import imaplib
import configparser
import re

In [41]:
def find_encoding_info(txt):
    info = email.header.decode_header(txt)
    s, encoding = info[0]
    return s, encoding


In [42]:
config = configparser.ConfigParser()
config.read('D:\개인\Work\config.ini')

['D:\\개인\\Work\\config.ini']

In [43]:
imap_url = imaplib.IMAP4_SSL('imap.gmail.com')

In [44]:
imap_url.login(config['Gmail']['id'], config['Gmail']['pw'])

('OK', [b'kimgs243@gmail.com authenticated (Success)'])

In [45]:
imap_url.select('INBOX')

('OK', [b'1'])

In [46]:
result, data = imap_url.search(None, 'ALL')

In [47]:
all_email = data[0].split()
print(all_email)

[b'1']


In [50]:
for mail in all_email:
    result, data = imap_url.fetch(mail, '(RFC822)')
    raw_email = data[0][1]
    raw_email_string = raw_email.decode('UTF-8')
    email_message = email.message_from_string(raw_email_string)

    # 메일 정보
    print('From: ', email_message['From'])
    print('Sender: ', email_message['Sender'])
    print('To: ', email_message['To'])
    print('Date: ', email_message['Date'])

    subject, encode = find_encoding_info(
        email_message['Subject'])
    print('Subject', subject)

    message = ''

    print('[Message]')

From:  GwiSoo Kim <kgs243@outlook.com>
Sender:  None
To:  GwiSoo Kim <kimgs243@gmail.com>
Date:  Tue, 20 Oct 2020 08:00:54 +0000
Subject FW: Atachfile's
[Message]


In [56]:
if email_message.is_multipart():
    for part in email_message.get_payload():
        if part.get_content_type() == 'text/plain':
            bytes = part.get_payload(decode=True)
            encode = part.get_content_charset()
            message = message + str(bytes, encode)
else:
    if email_message.get_content_type() == 'text/plain':
        bytes = email_message.get_payload(decode=True)
        encode = email_message.get_content_charset()
        message = str(bytes, encode)
print(message)

In [60]:
for part in email_message.walk():
    if part.get_content_maintype() == 'multipart':
        continue
    if part.get('Content-Disposition') is None:
        continue
    file_name = part.get_filename()

    # file_name = "".join(
    #     i for i in file_name if i not in "\/:*?<>|")

    # file_name = file_name.replace('UTF-8B', '')
    # file_name = file_name.replace('utf-8B', '')
    # print(file_name)

    if bool(file_name):
        file_path = os.path.join(
            'D:/개인/Work/Download', file_name)
        if not os.path.isfile(file_path):
            fp = open(file_path, 'wb')
            fp.write(part.get_payload(decode=True))
            fp.close()
else:
    continue

SyntaxError: 'continue' not properly in loop (<ipython-input-60-dabec65ac40d>, line 26)